https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# inputs = ((1x3)x5)
print(torch.randn(1,3).shape)


""" 
LSTM dimensions:

The first axis is the sequence itself.
The second indexes instances in the mini-batch.
The third indexes elements of the input. 
"""

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3), # hidden state 
          torch.randn(1, 1, 3)) # cell state  

for i in inputs: # going through each sequence where each sequence has 3 words 
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    print("shape: ", i.view(1, 1, -1).shape) # -> convert the 1x3->1x1x3
    out, hidden = lstm(i.view(1, 1, -1), hidden) # hidden state of the LSTM 


torch.Size([1, 3])
shape:  torch.Size([1, 1, 3])
shape:  torch.Size([1, 1, 3])
shape:  torch.Size([1, 1, 3])
shape:  torch.Size([1, 1, 3])
shape:  torch.Size([1, 1, 3])


In [11]:

# alternatively, we can do the entire sequence all at once.

# the first value returned by LSTM is all of the hidden states throughout the sequence. 
# the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.1309,  0.4328, -0.3984]],

        [[-0.0514,  0.4477,  0.1397]],

        [[-0.2294,  0.0784, -0.3161]],

        [[-0.0362,  0.3877, -0.1652]],

        [[-0.1349,  0.2108, -0.2128]]], grad_fn=<StackBackward0>)
(tensor([[[-0.1349,  0.2108, -0.2128]]], grad_fn=<StackBackward0>), tensor([[[-0.4776,  0.5837, -0.7949]]], grad_fn=<StackBackward0>))


In [26]:
https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [13]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [14]:
training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

In [15]:
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [22]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 10

In [23]:
""" 
LSTM dimensions:

The first axis is the sequence itself.
The second indexes instances in the mini-batch.
The third indexes elements of the input. 
"""

class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1)) # sequence_len x 1 x 10=hidden_dim - there will be 5=seq_len hidden layers
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1)) #in: 5x10 hidden: 10x3 out: 5x3
        tag_scores = F.log_softmax(tag_space, dim=1) #in 5x3 #out 5x3
        
        print("embeds: ", embeds.shape)
        print("lstm_out: ", lstm_out.shape)
        print("tag_space: ", tag_space.shape)
        print("tag_scores: ", tag_scores.shape)
        
        return tag_scores

In [24]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [27]:

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)


embeds:  torch.Size([5, 6])
lstm_out:  torch.Size([5, 1, 10])
tag_space:  torch.Size([5, 3])
tag_scores:  torch.Size([5, 3])
tensor([[-1.3326, -0.8771, -1.1387],
        [-1.3515, -0.8705, -1.1319],
        [-1.3289, -0.9208, -1.0875],
        [-1.2906, -0.8914, -1.1557],
        [-1.2828, -0.9596, -1.0797]])


In [30]:

for epoch in range(1):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        print("SHAPES FOR LOSS: ", tag_scores.shape, targets.shape)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()


embeds:  torch.Size([5, 6])
lstm_out:  torch.Size([5, 1, 10])
tag_space:  torch.Size([5, 3])
tag_scores:  torch.Size([5, 3])
SHAPES FOR LOSS:  torch.Size([5, 3]) torch.Size([5])
embeds:  torch.Size([4, 6])
lstm_out:  torch.Size([4, 1, 10])
tag_space:  torch.Size([4, 3])
tag_scores:  torch.Size([4, 3])
SHAPES FOR LOSS:  torch.Size([4, 3]) torch.Size([4])


In [31]:

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

embeds:  torch.Size([5, 6])
lstm_out:  torch.Size([5, 1, 10])
tag_space:  torch.Size([5, 3])
tag_scores:  torch.Size([5, 3])
tensor([[-1.2999, -0.8514, -1.2019],
        [-1.3227, -0.8409, -1.1964],
        [-1.3089, -0.8920, -1.1393],
        [-1.2398, -0.8715, -1.2301],
        [-1.2664, -0.9197, -1.1409]])
